# MuchLearningSuchWow - LSTM - Training

This notebook contains the code we used to define and train our LSTM network. The training code is based primarily on [this kernel](https://www.kaggle.com/bountyhunters/baseline-lstm-with-keras-0-7).

### Imports & Data Paths

In [1]:
import numpy as np
import pandas as pd
import pickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.utils import plot_model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
outputPath = "output/"
modelPath = "models/"

### Constants

In [3]:
timesteps = 14 # Number of previous days that will be used to predict the next day
startDay = 1000 # Number of days at start of data that will be ignored during training

### Loading Data

In [4]:
with open(outputPath + "/preprocessed_train_data.pkl", "rb") as f:
    df_train = pickle.load(f)

### Create Training Data and Labels

In [5]:
X_train = []
y_train = []
for i in range(timesteps, 1913 - startDay):
    X_train.append(df_train[i-timesteps:i])
    y_train.append(df_train[i][0:30490]) # Only use first 30490 columns (sales) as labels

In [6]:
del df_train

In [7]:
# Convert data to np array to be able to feed it to the model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(899, 14, 30491)
(899, 30490)


### LSTM Model

In [8]:
model = Sequential()

# 1D convolution layer
model.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="causal",activation="relu",input_shape=(X_train.shape[1], X_train.shape[2])))

# LSTM layers
layer_1_units=150
model.add(LSTM(units = layer_1_units, return_sequences = True))
model.add(Dropout(0.1))

layer_2_units=300
model.add(LSTM(units = layer_2_units, return_sequences = True))
model.add(Dropout(0.1))

layer_3_units=400
model.add(LSTM(units = layer_3_units))
model.add(Dropout(0.1))

# Output layer
model.add(Dense(units = 30490))

In [9]:
plot_model(model, modelPath + "/model.png")
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 14, 32)            6830016   
_________________________________________________________________
lstm_1 (LSTM)                (None, 14, 150)           109800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 150)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 14, 300)           541200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 400)               1121600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 400)              

### Training

In [10]:
# Compiling the model
model.compile(optimizer = Adam(learning_rate=0.001), loss = 'mean_squared_error')

# Fitting the model to the training set
nr_epochs = 50
batch_size = 16
model.fit(X_train, y_train, epochs = nr_epochs, batch_size = batch_size, verbose=1)

Epoch 1/50
899/899 [==============================] - 33s 37ms/step - loss: 0.0221
Epoch 2/50
899/899 [==============================] - 32s 36ms/step - loss: 0.0202
Epoch 3/50
899/899 [==============================] - 32s 35ms/step - loss: 0.0197
Epoch 4/50
899/899 [==============================] - 32s 35ms/step - loss: 0.0193
Epoch 5/50
899/899 [==============================] - 32s 35ms/step - loss: 0.0189
Epoch 6/50
899/899 [==============================] - 32s 36ms/step - loss: 0.0186
Epoch 7/50
899/899 [==============================] - 32s 36ms/step - loss: 0.0184
Epoch 8/50
899/899 [==============================] - 33s 37ms/step - loss: 0.0182
Epoch 9/50
899/899 [==============================] - 33s 37ms/step - loss: 0.0180
Epoch 10/50
899/899 [==============================] - 33s 37ms/step - loss: 0.0179
Epoch 11/50
899/899 [==============================] - 34s 37ms/step - loss: 0.0179
Epoch 12/50
899/899 [==============================] - 34s 38ms/step - loss: 0.0177
E

### Saving Result

In [11]:
model.save(modelPath + "/lstm_model")